## Example - Gaussian advection

This notebook is the v2.0.0 rewrite of the original ReMKiT1D_advection_test notebook, reproducing the example from Sections 3.2. and 5.1.1. of the ReMKiT1D code paper in v2.0.0 syntax. 

In [ ]:
%load_ext autoreload
%autoreload 2
import RMK_support as rmk
from RMK_support import node
from RMK_support.stencils import StaggeredDivStencil as Div, StaggeredGradStencil as Grad

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Context initialisation with IO and MPI context setting

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_advection_test/")
rk.mpiContext = rmk.MPIContext(numProcsX=4)

### Grid initialization


In [ ]:
# In normalized length or in meters - defaults to normalized unless isLengthInMeters=True in Grid
xGridWidths = 0.025*np.ones(512)
rk.grid = rmk.Grid(xGridWidths, interpretXGridAsWidths=True)

### Variable container


#### Adding variables


In [ ]:
nInit = 1 + np.exp(-(rk.grid.xGrid-np.mean(rk.grid.xGrid))**2) # A Gaussian perturbation
TInit = np.ones(len(rk.grid.xGrid)) # Constant temperature

n,n_dual = rmk.varAndDual("n",rk.grid,data=nInit) #both variable and its dual
T = rmk.Variable("T",rk.grid,data=TInit,isDerived=True,isCommunicated=False)
G_dual,G = rmk.varAndDual("G",rk.grid,primaryOnDualGrid=True) #the first return value is the primary, so here it is the dual

rk.variables.add(n,n_dual,T,G_dual,G)

In [ ]:
#v2.0.0 generating nodes from variables and vice versa
massRatio = 1/1836

W = rmk.varFromNode("dummyVar",rk.grid,node = 1.5*node(n)*node(T) + node(G)**2/(node(n)*massRatio)) 
rk.variables["W"] = W # this will copy and rename the variable to "W" when added 
rk.variables.add(W.rename("otherW")) # This is another way of doing it  

### Models 

$\frac{\partial n}{\partial t} = - \frac{\partial u}{\partial x}$

$m_i \frac{\partial u}{\partial t} = - \frac{\partial (nkT)}{\partial x}$

In v2.0.0, model construction is vastly simplified


In [ ]:
model = rmk.Model(name="adv")

# Models have ddt components, representing individual contributions to equations
# Matrix terms can be automatically constructed by invoking various stencils 

model.ddt[n] += - Div()(G_dual).rename("div_G") 
model.ddt[G_dual] += -massRatio/2 * Grad()(T * n).rename("grad_p") # Here n will be the implicit variable - in Matrix terms constructed by invoking stencils it is always the rightmost variable

rk.models.add(model)

### Integrator options

In [ ]:
# the implicit BDE integrator that checks convergence based on the variables n and G_dual
integrator = rmk.BDEIntegrator("BDE",nonlinTol=1e-12,absTol=10.0,convergenceVars=[n,G_dual])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) # Add all models in context
rk.integrationScheme = rmk.IntegrationScheme(dt=0.1,steps=integrationStep) #Create a scheme with our single step and a constant integration timestep 0.1
rk.integrationScheme.setFixedNumTimesteps(10000,200) # Run for 10000 steps outputting every 200

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Gaussian Advection Example")

### Create config file

In [ ]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset

### Compare with analytic solution

In [ ]:
times = dataset.coords['t'].data
wave_speed= np.sqrt(massRatio/2)
n_analytic=np.zeros((len(times),rk.grid.numX))
L = sum(xGridWidths)
for i in range(len(times)):
        leftPositionMod = (rk.grid.xGrid-wave_speed*times[i]) % L
        leftPosition = np.where(leftPositionMod > 0,leftPositionMod,leftPositionMod+L)
        rightPosition = (rk.grid.xGrid+wave_speed*times[i]) % L
        n_analytic[i,:] =1 + 0.5*(np.exp(-(leftPosition-np.mean(rk.grid.xGrid))**2) + np.exp(-(rightPosition-np.mean(rk.grid.xGrid))**2)) 


In [ ]:
dataName = 'n'

curveDict = {t: hv.Scatter(dataset[dataName][{"t":t}],label='simulation').opts(marker="o",color="r",s=6.0)*hv.Curve((rk.grid.xGrid,n_analytic[t,:]),label='analytic result').opts(title=f't = {dataset["t"].values[t]:.2f} '+dataset.coords["t"].attrs["units"],fontscale=2, fig_size=150,linewidth=3.0) for t in range(len(times))}
kdims = [hv.Dimension(('t', 'Time'),unit=dataset.coords["t"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()

### Check if W is calculated correctly

In [ ]:
dataName = 'W'

testWCalc = dataset['n']*dataset['T'] * 1.5 + dataset['G']**2 /( dataset['n']*massRatio) - dataset['W']

print(testWCalc.where(np.abs(testWCalc)>5e-16,drop=True))

### Reproducing graphs from the paper

### Relative error wrt analytic solution

In [ ]:
diff = np.abs(n_analytic - dataset['n'])/n_analytic

In [ ]:
relativeErrorPlot=hv.Curve(diff.reduce(np.max,'x')).opts(ylabel='$\delta n$',marker='o',fontscale=2, fig_size=150,linewidth=3.0)

In [ ]:
relativeErrorPlot.opts()

In [ ]:
hv.output(fig='pdf')
hv.save(relativeErrorPlot, 'advectionTestRelErr.pdf', dpi=144)

### Final simulation state

In [ ]:
hv.save(curveDict[50].opts(legend_position='top',legend_cols=1,title=''),'finalDensityAdv.pdf',dpi=144)